### Задание:
Необходимо построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.
Вывести качество полученных моделей на тестовой выборке, используя функцию score у модели.

Описание переменных:

* age - возраст, непрерывная величина
* workclass - рабочий класс, категориальная переменная
* fnlwgt - (final weight) - примерная оценка количества людей, которое представляет каждая строка данных, непрерывная величина
* education - уровень образования, категориальная величина
* education-num - количество лет обучения, непрерывная величина
* marital-status - семейный статус, ктегориальная переменная
* occupation - род занятий, категориальная переменная
* relationship - статус отношений, категориальная переменная
* race - расса, категориальная переменная
* sex - пол, категориальная переменная
* capital-gain - прирост капитала, непрерывная величина
* capital-loss - убыль капитала, непрерывная величина
* hours-per-week - количество рабочих часов в неделю, непрерывная величина
* native-country - родная страна, категориальная переменная

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [2]:
income = pd.read_csv('adult.csv', na_values = '?')
income.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
# Краткая информация о датафрейме
income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        46043 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       46033 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   47985 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [4]:
# Определение наиболее частых значений для переменных с пропусками
income[['workclass', 'occupation', 'native-country']].describe()

,workclass,occupation,native-country
count,46043,46033,47985
unique,8,14,41
top,Private,Prof-specialty,United-States
freq,33906,6172,43832


In [5]:
# Замена пропусков наиболее часто встречающимися значениями
income['workclass'].fillna('Private', inplace = True)
income['occupation'].fillna('Prof-specialty', inplace = True)
income['native-country'].fillna('United-States', inplace = True)

In [6]:
# Вывод соответствия переменных education и educational-num
for ed in income.education.unique():
    income_ = income[income.education == ed]
    print(ed, income_['educational-num'].unique())

11th [7]
HS-grad [9]
Assoc-acdm [12]
Some-college [10]
10th [6]
Prof-school [15]
7th-8th [4]
Bachelors [13]
Masters [14]
Doctorate [16]
5th-6th [3]
Assoc-voc [11]
9th [5]
12th [8]
1st-4th [2]
Preschool [1]


Пропуски значений есть только в категориальных переменных: workclass, occupation, native-country. Имеющиес пропуски заполнены наиболее часто встречающимися значениями переменной.

Каждому значению переменной education соответсвует только одно значение количества лет обучения, т.е. столбец educational-num дублируется переменную education. Для обучения модели будет использован только один из них.

Переменная fnlwgt является оценочной и не влияет на целевую переменную. В расчете модели использоваться так же не будет.

In [7]:
# Отбор переменных для обучения модели
selected_feature = income[['age', 'educational-num', 'capital-gain', 'capital-loss', 'hours-per-week',
                          'workclass', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country']]

# Перевод категориальных переменных в бинарный систему
X = pd.get_dummies(selected_feature, columns = ['workclass', 'marital-status', 'occupation',
                                                      'relationship', 'race', 'gender','native-country'])

In [8]:
# Перевод целевой переменной в бинарную систему

# Создание объекта Label Encoder
le = LabelEncoder()

# Обучение объекта
le.fit_transform(income.income)

# Преобразование целевой переменной в объект Series
y = pd.Series(le.fit_transform(income.income))

### Модель логистической регрессии

In [9]:
# Создание пайплайна
model_logreg = make_pipeline(StandardScaler(), LogisticRegression())

# Разделение выборки на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

# Обучение модели
model_logreg.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [10]:
# Точночть предсказания обучаюшей выбороки
model_logreg.score(X_train, y_train)

0.849776243821112

In [11]:
# Точночть предсказания тестовой выбороки
model_logreg.score(X_test, y_test)

0.8574353374735549

### Модель SVC

In [12]:
# Создание пайплайна
model_SVC = make_pipeline(StandardScaler(), SVC(gamma = 'auto'))

# Разделение выборки на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [13]:
# Обучение модели
model_SVC.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [14]:
# Точночть предсказания обучаюшей выбороки
model_SVC.score(X_train, y_train)

0.8567082979905818

In [15]:
# Точночть предсказания обучаюшей выбороки
model_SVC.score(X_test, y_test)

0.8529311403808094

Вывод:

Обе модели имеют близкие показатели точноти предсказания - около 85%. На модели логистической регрессии точность достигает 85.74%, т.е. выше чем при применении метода опорных векторов.
При этом точность предсказания логистической регрессии на тестовых данных выше, чем на обучающей выборке.